## Set path

In [1]:
import os
os.chdir("code/cTP-net/")
datasets_dir = os.path.abspath(os.path.join(os.getcwd(), '../../datasets'))
outputs_dir = os.path.abspath(os.path.join(os.getcwd(), '../../outputs'))

## Load necessary libraries

In [2]:
from train import training, predicting
import torch
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.stats.mstats import gmean

/home/hongyj/.conda/envs/ctpnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [3]:
train_data = sc.read_h5ad(os.path.join(datasets_dir, "different samples/CITE-SLN111-Gayoso/Mouse1.h5ad"))
test_data = sc.read_h5ad(os.path.join(datasets_dir, "different samples/CITE-SLN111-Gayoso/Mouse2.h5ad"))
train_data, test_data

(AnnData object with n_obs × n_vars = 9264 × 13553
     obs: 'n_protein_counts', 'n_proteins', 'seurat_hash_id', 'batch_indices', 'hash_id', 'n_genes', 'percent_mito', 'leiden_subclusters', 'cell_types'
     var: 'gene_ids', 'feature_types', 'highly_variable', 'highly_variable_mean_variance', 'encode', 'hvg_encode'
     uns: 'protein_name', 'version'
     obsm: 'protein_expression',
 AnnData object with n_obs × n_vars = 7564 × 13553
     obs: 'n_protein_counts', 'n_proteins', 'seurat_hash_id', 'batch_indices', 'hash_id', 'n_genes', 'percent_mito', 'leiden_subclusters', 'cell_types'
     var: 'gene_ids', 'feature_types', 'highly_variable', 'highly_variable_mean_variance', 'encode', 'hvg_encode'
     uns: 'protein_name', 'version'
     obsm: 'protein_expression')

## Load SAVER-X denoised gene expression data

In [4]:
train_denoised_gene_expression_data = pd.read_csv(
    os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/Mouse1_denoised_gene_expression_data.txt"), delimiter=" ").T
test_denoised_gene_expression_data = pd.read_csv(
    os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/Mouse2_denoised_gene_expression_data.txt"), delimiter=" ").T
train_denoised_gene_expression_data, test_denoised_gene_expression_data

(                    Mrpl15  Lypla1  Tcea1  Atp6v1h  Rb1cc1  4732440D04Rik  \
 AAACCCAAGGGTAATT-1   0.562   0.155  0.403    0.277   0.409          0.149   
 AAACCCAAGGTAAACT-1   0.481   0.159  0.329    0.227   0.341          0.099   
 AAACCCACACTAGGTT-1   0.390   0.166  0.561    0.243   0.645          0.162   
 AAACCCACAGATACCT-1   0.519   0.164  0.452    0.205   0.272          0.113   
 AAACCCACAGGAATAT-1   0.370   0.211  0.307    0.198   0.732          0.138   
 ...                    ...     ...    ...      ...     ...            ...   
 TTTGTTGGTAGTCACT-1   0.397   0.260  0.323    0.181   0.407          0.136   
 TTTGTTGGTGGTAATA-1   0.275   0.165  0.382    0.190   0.630          0.179   
 TTTGTTGGTTTACGTG-1   0.224   0.164  0.381    0.197   0.801          0.218   
 TTTGTTGTCATGAAAG-1   0.444   0.165  0.280    0.183   0.526          0.087   
 TTTGTTGTCTCCGATC-1   0.462   0.215  0.389    0.218   0.553          0.187   
 
                     Pcmtd1  Gm26901   Rrs1  Adhfe1  ...  Tmlh

## Preprocess gene expression data

In [5]:
train_data.X = train_denoised_gene_expression_data.values
test_data.X = test_denoised_gene_expression_data.values

sc.pp.normalize_total(train_data,target_sum=1e4)
sc.pp.normalize_total(test_data,target_sum=1e4)

norm_train_rna = train_data.X
norm_test_rna = test_data.X

norm_train_rna = np.log1p(norm_train_rna)
norm_test_rna = np.log1p(norm_test_rna)

## Preprocess protein expression data

In [6]:
train_protein = pd.DataFrame(train_data.obsm["protein_expression"].todense(), index=train_data.obs.index, columns=train_data.uns["protein_name"]) + 1
test_protein = pd.DataFrame(test_data.obsm["protein_expression"].todense(), index=test_data.obs.index, columns=test_data.uns["protein_name"]) + 1

gpc_train = gmean(train_protein,axis=1)
gpc_test = gmean(test_protein,axis=1)

train_protein_clr = np.log((train_protein) / gpc_train.reshape(-1,1))
test_protein_clr = np.log((test_protein) / gpc_test.reshape(-1,1))

## Save preprocessed protein expression data for evaluation

In [7]:
test_protein_clr.to_csv(os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/cTP-net/test_protein_clr.txt"), sep="\t")

## Adjust protein names

In [8]:
protein_names = test_protein_clr.columns
protein_names = [i.replace(".","/") for i in protein_names]
cell_names = test_protein_clr.index

## Train model

In [9]:
shared_generator, protein_predictor = training(source_rna=norm_train_rna, source_protein=train_protein_clr.values, protein_name=protein_names)

[gener Epoch 0/139] [gener Batch 0/93] [gener L1 loss: 1.129127] 
[gener Epoch 0/139] [gener Batch 1/93] [gener L1 loss: 1.100358] 
[gener Epoch 0/139] [gener Batch 2/93] [gener L1 loss: 1.058774] 
[gener Epoch 0/139] [gener Batch 3/93] [gener L1 loss: 1.040962] 
[gener Epoch 0/139] [gener Batch 4/93] [gener L1 loss: 0.994473] 
[gener Epoch 0/139] [gener Batch 5/93] [gener L1 loss: 0.980968] 
[gener Epoch 0/139] [gener Batch 6/93] [gener L1 loss: 0.986839] 
[gener Epoch 0/139] [gener Batch 7/93] [gener L1 loss: 0.967693] 
[gener Epoch 0/139] [gener Batch 8/93] [gener L1 loss: 0.942676] 
[gener Epoch 0/139] [gener Batch 9/93] [gener L1 loss: 0.930529] 
[gener Epoch 0/139] [gener Batch 10/93] [gener L1 loss: 0.929385] 
[gener Epoch 0/139] [gener Batch 11/93] [gener L1 loss: 0.908510] 
[gener Epoch 0/139] [gener Batch 12/93] [gener L1 loss: 0.916641] 
[gener Epoch 0/139] [gener Batch 13/93] [gener L1 loss: 0.911695] 
[gener Epoch 0/139] [gener Batch 14/93] [gener L1 loss: 0.894915] 
[gene

[gener Epoch 1/139] [gener Batch 30/93] [gener L1 loss: 0.616751] 
[gener Epoch 1/139] [gener Batch 31/93] [gener L1 loss: 0.624576] 
[gener Epoch 1/139] [gener Batch 32/93] [gener L1 loss: 0.614997] 
[gener Epoch 1/139] [gener Batch 33/93] [gener L1 loss: 0.614065] 
[gener Epoch 1/139] [gener Batch 34/93] [gener L1 loss: 0.613326] 
[gener Epoch 1/139] [gener Batch 35/93] [gener L1 loss: 0.612933] 
[gener Epoch 1/139] [gener Batch 36/93] [gener L1 loss: 0.601237] 
[gener Epoch 1/139] [gener Batch 37/93] [gener L1 loss: 0.623086] 
[gener Epoch 1/139] [gener Batch 38/93] [gener L1 loss: 0.619116] 
[gener Epoch 1/139] [gener Batch 39/93] [gener L1 loss: 0.615459] 
[gener Epoch 1/139] [gener Batch 40/93] [gener L1 loss: 0.605515] 
[gener Epoch 1/139] [gener Batch 41/93] [gener L1 loss: 0.608519] 
[gener Epoch 1/139] [gener Batch 42/93] [gener L1 loss: 0.610643] 
[gener Epoch 1/139] [gener Batch 43/93] [gener L1 loss: 0.599873] 
[gener Epoch 1/139] [gener Batch 44/93] [gener L1 loss: 0.6029

[gener Epoch 2/139] [gener Batch 60/93] [gener L1 loss: 0.493697] 
[gener Epoch 2/139] [gener Batch 61/93] [gener L1 loss: 0.486915] 
[gener Epoch 2/139] [gener Batch 62/93] [gener L1 loss: 0.501884] 
[gener Epoch 2/139] [gener Batch 63/93] [gener L1 loss: 0.492335] 
[gener Epoch 2/139] [gener Batch 64/93] [gener L1 loss: 0.488395] 
[gener Epoch 2/139] [gener Batch 65/93] [gener L1 loss: 0.486724] 
[gener Epoch 2/139] [gener Batch 66/93] [gener L1 loss: 0.482639] 
[gener Epoch 2/139] [gener Batch 67/93] [gener L1 loss: 0.481855] 
[gener Epoch 2/139] [gener Batch 68/93] [gener L1 loss: 0.488914] 
[gener Epoch 2/139] [gener Batch 69/93] [gener L1 loss: 0.479481] 
[gener Epoch 2/139] [gener Batch 70/93] [gener L1 loss: 0.501090] 
[gener Epoch 2/139] [gener Batch 71/93] [gener L1 loss: 0.495139] 
[gener Epoch 2/139] [gener Batch 72/93] [gener L1 loss: 0.484136] 
[gener Epoch 2/139] [gener Batch 73/93] [gener L1 loss: 0.484353] 
[gener Epoch 2/139] [gener Batch 74/93] [gener L1 loss: 0.4877

[gener Epoch 3/139] [gener Batch 90/93] [gener L1 loss: 0.445764] 
[gener Epoch 3/139] [gener Batch 91/93] [gener L1 loss: 0.450620] 
[gener Epoch 3/139] [gener Batch 92/93] [gener L1 loss: 0.446836] 
[gener Epoch 4/139] [gener Batch 0/93] [gener L1 loss: 0.426822] 
[gener Epoch 4/139] [gener Batch 1/93] [gener L1 loss: 0.449088] 
[gener Epoch 4/139] [gener Batch 2/93] [gener L1 loss: 0.447805] 
[gener Epoch 4/139] [gener Batch 3/93] [gener L1 loss: 0.437531] 
[gener Epoch 4/139] [gener Batch 4/93] [gener L1 loss: 0.445555] 
[gener Epoch 4/139] [gener Batch 5/93] [gener L1 loss: 0.441587] 
[gener Epoch 4/139] [gener Batch 6/93] [gener L1 loss: 0.433544] 
[gener Epoch 4/139] [gener Batch 7/93] [gener L1 loss: 0.433242] 
[gener Epoch 4/139] [gener Batch 8/93] [gener L1 loss: 0.448134] 
[gener Epoch 4/139] [gener Batch 9/93] [gener L1 loss: 0.427430] 
[gener Epoch 4/139] [gener Batch 10/93] [gener L1 loss: 0.444406] 
[gener Epoch 4/139] [gener Batch 11/93] [gener L1 loss: 0.443800] 
[gene

[gener Epoch 5/139] [gener Batch 29/93] [gener L1 loss: 0.438051] 
[gener Epoch 5/139] [gener Batch 30/93] [gener L1 loss: 0.432219] 
[gener Epoch 5/139] [gener Batch 31/93] [gener L1 loss: 0.443461] 
[gener Epoch 5/139] [gener Batch 32/93] [gener L1 loss: 0.435078] 
[gener Epoch 5/139] [gener Batch 33/93] [gener L1 loss: 0.432626] 
[gener Epoch 5/139] [gener Batch 34/93] [gener L1 loss: 0.434912] 
[gener Epoch 5/139] [gener Batch 35/93] [gener L1 loss: 0.441617] 
[gener Epoch 5/139] [gener Batch 36/93] [gener L1 loss: 0.437743] 
[gener Epoch 5/139] [gener Batch 37/93] [gener L1 loss: 0.444414] 
[gener Epoch 5/139] [gener Batch 38/93] [gener L1 loss: 0.441686] 
[gener Epoch 5/139] [gener Batch 39/93] [gener L1 loss: 0.432673] 
[gener Epoch 5/139] [gener Batch 40/93] [gener L1 loss: 0.431418] 
[gener Epoch 5/139] [gener Batch 41/93] [gener L1 loss: 0.445627] 
[gener Epoch 5/139] [gener Batch 42/93] [gener L1 loss: 0.445483] 
[gener Epoch 5/139] [gener Batch 43/93] [gener L1 loss: 0.4406

...

[gener Epoch 133/139] [gener Batch 53/93] [gener L1 loss: 0.372832] 
[gener Epoch 133/139] [gener Batch 54/93] [gener L1 loss: 0.369559] 
[gener Epoch 133/139] [gener Batch 55/93] [gener L1 loss: 0.367162] 
[gener Epoch 133/139] [gener Batch 56/93] [gener L1 loss: 0.370187] 
[gener Epoch 133/139] [gener Batch 57/93] [gener L1 loss: 0.361817] 
[gener Epoch 133/139] [gener Batch 58/93] [gener L1 loss: 0.368797] 
[gener Epoch 133/139] [gener Batch 59/93] [gener L1 loss: 0.367209] 
[gener Epoch 133/139] [gener Batch 60/93] [gener L1 loss: 0.371093] 
[gener Epoch 133/139] [gener Batch 61/93] [gener L1 loss: 0.368301] 
[gener Epoch 133/139] [gener Batch 62/93] [gener L1 loss: 0.381709] 
[gener Epoch 133/139] [gener Batch 63/93] [gener L1 loss: 0.370159] 
[gener Epoch 133/139] [gener Batch 64/93] [gener L1 loss: 0.370029] 
[gener Epoch 133/139] [gener Batch 65/93] [gener L1 loss: 0.364557] 
[gener Epoch 133/139] [gener Batch 66/93] [gener L1 loss: 0.367289] 
[gener Epoch 133/139] [gener Batch

[gener Epoch 134/139] [gener Batch 80/93] [gener L1 loss: 0.372118] 
[gener Epoch 134/139] [gener Batch 81/93] [gener L1 loss: 0.373119] 
[gener Epoch 134/139] [gener Batch 82/93] [gener L1 loss: 0.367093] 
[gener Epoch 134/139] [gener Batch 83/93] [gener L1 loss: 0.367405] 
[gener Epoch 134/139] [gener Batch 84/93] [gener L1 loss: 0.374416] 
[gener Epoch 134/139] [gener Batch 85/93] [gener L1 loss: 0.374192] 
[gener Epoch 134/139] [gener Batch 86/93] [gener L1 loss: 0.371881] 
[gener Epoch 134/139] [gener Batch 87/93] [gener L1 loss: 0.366400] 
[gener Epoch 134/139] [gener Batch 88/93] [gener L1 loss: 0.373509] 
[gener Epoch 134/139] [gener Batch 89/93] [gener L1 loss: 0.366621] 
[gener Epoch 134/139] [gener Batch 90/93] [gener L1 loss: 0.369798] 
[gener Epoch 134/139] [gener Batch 91/93] [gener L1 loss: 0.377005] 
[gener Epoch 134/139] [gener Batch 92/93] [gener L1 loss: 0.356130] 
[gener Epoch 135/139] [gener Batch 0/93] [gener L1 loss: 0.363133] 
[gener Epoch 135/139] [gener Batch 

[gener Epoch 136/139] [gener Batch 14/93] [gener L1 loss: 0.376412] 
[gener Epoch 136/139] [gener Batch 15/93] [gener L1 loss: 0.357059] 
[gener Epoch 136/139] [gener Batch 16/93] [gener L1 loss: 0.368809] 
[gener Epoch 136/139] [gener Batch 17/93] [gener L1 loss: 0.371379] 
[gener Epoch 136/139] [gener Batch 18/93] [gener L1 loss: 0.361533] 
[gener Epoch 136/139] [gener Batch 19/93] [gener L1 loss: 0.370176] 
[gener Epoch 136/139] [gener Batch 20/93] [gener L1 loss: 0.370969] 
[gener Epoch 136/139] [gener Batch 21/93] [gener L1 loss: 0.369039] 
[gener Epoch 136/139] [gener Batch 22/93] [gener L1 loss: 0.359977] 
[gener Epoch 136/139] [gener Batch 23/93] [gener L1 loss: 0.359228] 
[gener Epoch 136/139] [gener Batch 24/93] [gener L1 loss: 0.369434] 
[gener Epoch 136/139] [gener Batch 25/93] [gener L1 loss: 0.364296] 
[gener Epoch 136/139] [gener Batch 26/93] [gener L1 loss: 0.374392] 
[gener Epoch 136/139] [gener Batch 27/93] [gener L1 loss: 0.363090] 
[gener Epoch 136/139] [gener Batch

[gener Epoch 137/139] [gener Batch 41/93] [gener L1 loss: 0.377198] 
[gener Epoch 137/139] [gener Batch 42/93] [gener L1 loss: 0.373773] 
[gener Epoch 137/139] [gener Batch 43/93] [gener L1 loss: 0.373108] 
[gener Epoch 137/139] [gener Batch 44/93] [gener L1 loss: 0.370009] 
[gener Epoch 137/139] [gener Batch 45/93] [gener L1 loss: 0.368298] 
[gener Epoch 137/139] [gener Batch 46/93] [gener L1 loss: 0.367228] 
[gener Epoch 137/139] [gener Batch 47/93] [gener L1 loss: 0.372488] 
[gener Epoch 137/139] [gener Batch 48/93] [gener L1 loss: 0.367937] 
[gener Epoch 137/139] [gener Batch 49/93] [gener L1 loss: 0.369892] 
[gener Epoch 137/139] [gener Batch 50/93] [gener L1 loss: 0.372273] 
[gener Epoch 137/139] [gener Batch 51/93] [gener L1 loss: 0.377985] 
[gener Epoch 137/139] [gener Batch 52/93] [gener L1 loss: 0.368599] 
[gener Epoch 137/139] [gener Batch 53/93] [gener L1 loss: 0.372031] 
[gener Epoch 137/139] [gener Batch 54/93] [gener L1 loss: 0.368836] 
[gener Epoch 137/139] [gener Batch

[gener Epoch 138/139] [gener Batch 68/93] [gener L1 loss: 0.370835] 
[gener Epoch 138/139] [gener Batch 69/93] [gener L1 loss: 0.371546] 
[gener Epoch 138/139] [gener Batch 70/93] [gener L1 loss: 0.379697] 
[gener Epoch 138/139] [gener Batch 71/93] [gener L1 loss: 0.373671] 
[gener Epoch 138/139] [gener Batch 72/93] [gener L1 loss: 0.369010] 
[gener Epoch 138/139] [gener Batch 73/93] [gener L1 loss: 0.365987] 
[gener Epoch 138/139] [gener Batch 74/93] [gener L1 loss: 0.371674] 
[gener Epoch 138/139] [gener Batch 75/93] [gener L1 loss: 0.373277] 
[gener Epoch 138/139] [gener Batch 76/93] [gener L1 loss: 0.383915] 
[gener Epoch 138/139] [gener Batch 77/93] [gener L1 loss: 0.384999] 
[gener Epoch 138/139] [gener Batch 78/93] [gener L1 loss: 0.374480] 
[gener Epoch 138/139] [gener Batch 79/93] [gener L1 loss: 0.373426] 
[gener Epoch 138/139] [gener Batch 80/93] [gener L1 loss: 0.371462] 
[gener Epoch 138/139] [gener Batch 81/93] [gener L1 loss: 0.372443] 
[gener Epoch 138/139] [gener Batch

## Save model

In [10]:
torch.save(shared_generator, os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/cTP-net/shared_generator.pkl"))
torch.save(protein_predictor, os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/cTP-net/protein_predictor.pkl"))

## Impute protein expression data in test

In [11]:
test_protein_predition = predicting(shared_generator=shared_generator, protein_predictor=protein_predictor, target_rna=norm_test_rna, 
                                    protein_name=protein_names)

## Save prediction

In [12]:
test_protein_predition = pd.DataFrame(test_protein_predition, columns=protein_names, index=test_protein_clr.index)
test_protein_predition.to_csv(os.path.join(outputs_dir, "different samples/CITE-SLN111-Gayoso-Mouse1toMouse2/cTP-net/test_protein_prediction.txt"), sep="\t")
test_protein_predition

,ADT_CD102_A0104,ADT_CD103_A0201,ADT_CD106_A0226,ADT_CD115(CSF-1R)_A0105,ADT_CD117(c-Kit)_A0012,ADT_CD11a_A0595,ADT_CD11c_A0106,ADT_CD122(IL-2Rb)_A0227,ADT_CD127(IL-7Ra)_A0198,ADT_CD134(OX-40)_A0195,...,ADT_TCRVr1/1-Cr4_A0209,ADT_TCRVr2_A0211,ADT_TCRVr3_A0210,ADT_TCRbchain_A0120,ADT_TCRr-d_A0121,ADT_TER-119-ErythroidCells_A0122,ADT_Tim-4_A0567,ADT_XCR1_A0568,ADT_anti-P2RY12_A0415,ADT_integrinb7_A0214
index,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAATCTAG-2,1.615258,0.025867,-0.344342,-0.798843,-0.643836,1.438133,-0.991613,-0.742953,-0.910569,0.449824,...,-1.613728,-1.332615,0.015135,-0.452836,-0.473754,-1.361985,-0.318915,-0.710373,-0.740200,1.429022
AAACCCACACCGGAAA-2,2.397919,0.003684,0.007041,-0.571293,-0.383403,1.367856,-0.906420,-0.800095,-1.196170,0.566871,...,-1.433849,-1.285789,0.242834,-0.456209,-0.569097,-1.453604,0.059059,-0.684711,-0.371998,0.925742
AAACCCACACTACTTT-2,1.986849,-0.152050,-0.414074,-0.606106,-0.969298,2.054199,-0.916289,-0.889699,-1.292365,0.588226,...,-1.769940,-1.343223,-0.058091,-0.681636,-0.647330,-1.876969,-0.250250,-0.727606,-0.468123,1.391037
AAACCCAGTAGGCAAC-2,1.991563,-0.121896,-0.394430,-0.660839,-0.841756,1.992350,-0.875666,-0.920071,-1.185853,0.522659,...,-1.799600,-1.366791,-0.054192,-0.515686,-0.726396,-1.903375,-0.219799,-0.752328,-0.619701,1.287008
AAACCCAGTCTCAGGC-2,1.923694,-0.067515,-0.410641,-0.728470,-0.736186,2.052947,-1.082852,-0.851845,-1.357699,0.576373,...,-1.752512,-1.355120,-0.000947,-0.590239,-0.619564,-1.775051,-0.115073,-0.691173,-0.679556,1.546012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTAAGATCA-2,1.987110,0.083375,-0.608646,-0.717406,-0.853735,2.019354,-0.965294,-1.001490,-1.228369,0.647306,...,-1.857099,-1.186820,-0.029823,-0.435512,-0.627666,-1.869496,-0.268238,-0.604115,-0.648222,1.539528
TTTGTTGGTCAGACTT-2,1.708909,0.316016,-0.512173,-0.541135,-0.233382,2.951688,-1.028624,-1.048359,-0.248723,0.491536,...,-1.556669,-1.634656,0.214475,2.387301,-0.601083,-1.641431,-0.124634,-0.741210,-0.441564,1.436717
TTTGTTGGTGTTTACG-2,2.440383,-0.153233,-0.204975,-0.577883,-0.681982,2.109923,-0.862487,-1.124173,-1.292636,0.549816,...,-1.868568,-1.562418,-0.008475,-0.370206,-0.840610,-1.847256,-0.028497,-0.822257,-0.909263,0.789215
